In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import lightning as L

from lightning.pytorch.callbacks import ModelCheckpoint

from src.model.modeling_char_encoder import LitCharEncoder
from src.data.dataset import get_train_dataloader, get_dev_dataloader, get_test_dataloader

/home/jjw1214/.conda/envs/jjw1214_py312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
SEED=42
DATASET_NAME = 'jwengr/C-LLM'
SPACE_TOKEN = '[SEP]'
UNK_TOKEN = '[UNK]'
PAD_TOKEN = '[PAD]'
MINI_BATCH_SIZE=32
N_BATCH = 1
BASE_MODEL_NAME='google-bert/bert-base-multilingual-cased'
EPOCHS=10
LEARNING_RATE = 5e-5
TRAIN_MAX_LENGTH=128
VALID_MAX_LENGTH=128
INFERENCE_SENTENCE_MAX_LENGTH=64
INFERENCE_SENTENCE_MIN_LENGTH=32
INFERENCE_SENTENCE_N_OVERLAP=3

In [5]:
L.seed_everything(SEED)

Seed set to 42


42

In [6]:
train_dl = get_train_dataloader(DATASET_NAME, batch_size=MINI_BATCH_SIZE, max_length=TRAIN_MAX_LENGTH)
dev_dl = get_dev_dataloader(DATASET_NAME, batch_size=MINI_BATCH_SIZE, max_length=VALID_MAX_LENGTH)
test_dl = get_test_dataloader(DATASET_NAME, batch_size=1)

In [7]:
lit_char_encoder = LitCharEncoder(
    base_model_name=BASE_MODEL_NAME,
    space_token=SPACE_TOKEN,
    unk_token=UNK_TOKEN,
    pad_token=PAD_TOKEN,
    lr=LEARNING_RATE,
    epochs=EPOCHS,
    inference_sentence_max_length=INFERENCE_SENTENCE_MAX_LENGTH,
    inference_sentence_min_length=INFERENCE_SENTENCE_MIN_LENGTH,
    inference_sentence_n_overlap=INFERENCE_SENTENCE_N_OVERLAP
)

/home/jjw1214/.conda/envs/jjw1214_py312/lib/python3.12/site-packages/transformers/models/auto/modeling_auto.py:2178: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at google-bert/bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be e

In [8]:
checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints/charencoder',
    filename=f"{DATASET_NAME.split('/')[1]}/{BASE_MODEL_NAME.split('/')[1]}"+"-{epoch:02d}-{valid_loss:.4f}",
    every_n_epochs=1,
    save_top_k=-1,
)

In [9]:
trainer = L.Trainer(
    callbacks=[checkpoint_callback],
    precision='bf16',
    max_epochs=EPOCHS,
    enable_checkpointing=True,
    accumulate_grad_batches=N_BATCH
)

/home/jjw1214/.conda/envs/jjw1214_py312/lib/python3.12/site-packages/lightning/fabric/connector.py:571: `precision=bf16` is supported for historical reasons but its usage is discouraged. Please set your precision to bf16-mixed instead!
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(lit_char_encoder, train_dl, dev_dl)

You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]
/home/jjw1214/.conda/envs/jjw1214_py312/lib/python3.12/site-packages/lightning/pytorch/utilities/model_summary/model_summary.py:231: Precision bf16-mixed is not supported by the model summary.  Estimated model size in MB will not be accurate. Using 32 bits instead.

  | Name    | Type        | Params | Mode 
------------------------------------------------
0 | encoder | CharEncoder | 177 M  | train
------------------------------------------------
177 M     Trainable params
0         Non-trainable params
177 M     Total params
711.898   Total estimated model params size (MB)
1         Modul

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/jjw1214/.conda/envs/jjw1214_py312/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


/home/jjw1214/.conda/envs/jjw1214_py312/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


Epoch 5:  92%|█████████▏| 9483/10354 [18:36<01:42,  8.49it/s, v_num=53] 

In [7]:
lit_char_encoder = LitCharEncoder.load_from_checkpoint(
    'checkpoints/charencoder/C-LLM/bert-base-multilingual-cased-epoch=00-valid_loss=0.0197.ckpt',
    base_model_name=BASE_MODEL_NAME,
    space_token=SPACE_TOKEN,
    unk_token=UNK_TOKEN,
    pad_token=PAD_TOKEN,
    lr=LEARNING_RATE,
    epochs=EPOCHS,
    inference_sentence_max_length=INFERENCE_SENTENCE_MAX_LENGTH,
    inference_sentence_min_length=INFERENCE_SENTENCE_MIN_LENGTH,
    inference_sentence_n_overlap=INFERENCE_SENTENCE_N_OVERLAP
)

/home/jjw1214/.conda/envs/jjw1214_py312/lib/python3.12/site-packages/transformers/models/auto/modeling_auto.py:2178: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
Some weights of the model checkpoint at google-bert/bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be e

In [8]:
trainer = L.Trainer()

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [9]:
preds = trainer.predict(lit_char_encoder, test_dl)

You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]
/home/jjw1214/.conda/envs/jjw1214_py312/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:433: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


Predicting DataLoader 0:  10%|█         | 2785/26731 [00:22<03:10, 125.39it/s]​说到这,车德钢觉得有必要为大家进行一下简单的科普,越野车上的所谓“差速锁”对于越野性能究竟有着怎样的提升。
说到这，车德钢觉得有必要为大家进行一下简单的科普,越野车上的所谓[差速锁[对于越野性能究竟有着怎样的提升，
Predicting DataLoader 0:  11%|█         | 2807/26731 [00:22<03:10, 125.37it/s]操控器方面,方向盘比较轻,灵敏度也高,转向也非常的精准,天气冷的话还可以提前进行远程开启空调,车内的中控屏是方便,特别的智能化,车内的360全景影像画面清晰,开起来信心非常的强,上手也比较快。
操控器方面，方向盘比较轻,灵敏度也高,转向也非常的精准，天气冷的话还可以提前进行远程开启空调,车内的中控屏是方便,特别的智能化,车内的360全景影像画面清晰,开起来信心非常的强,上手也比较快，
Predicting DataLoader 0:  13%|█▎        | 3346/26731 [00:26<03:06, 125.17it/s]六、合作生效的标准乙方将此页面打萤签字、盖章后邮政寄给甲方。
六、合作生效的标准乙方将此页面打黑签字、盖章后邮政寄给甲方，
Predicting DataLoader 0:  13%|█▎        | 3401/26731 [00:27<03:06, 125.29it/s]2)乙方有权获得甲方市畅宣传广告上的支持。
2)乙方有权获得甲方市场宣传广告上的支持，
Predicting DataLoader 0:  14%|█▍        | 3717/26731 [00:29<03:02, 125.96it/s]⑤每月财务,由甲方保管,乙方监管,每月核算签字后,分红。
⑤每月财务,由甲方保管,乙方监管,每月核算签字后,分红，
Predicting DataLoader 0:  14%|█▍        | 3807/26731 [00:30<03:01, 126.07it/s]*本合同含产品报价清单一份,经双方签字盖章确认,与合同正文具有相同的法律效力;
*本合同含产品报价清单

In [11]:
for batch in test_dl:
    break

In [12]:
batch

{'sentence': ['在最近的福特资本市场日投资者介绍会上,首席产品平台和运营官'],
 'sentence_noisy': ['在最近的福特资本市场日投资者介绍会上,首席产品平台和运营干'],
 'category': ['car']}

In [10]:
preds[0]

['在最近的福特资本市场日投资者介绍会上，首席产品平台和运营干']